In [ ]:
!pip3 install vtk
!pip3 install nipype

In [ ]:
import os
import re
import vtk
import sys
import csv
import math
import shutil
import numpy as np
import pandas as pd
import nibabel as nib
from pathlib import Path
from sklearn import metrics
import plotly.express as px
import matplotlib.pyplot as plt
from nipype.interfaces import fsl
import plotly.graph_objects as go
from sklearn.cluster import DBSCAN
from nipype.testing import example_data
from mpl_toolkits.mplot3d import Axes3D
from google.colab import files  


# defining nii to stl conversion

def nii_2_mesh(filename_nii, filename_stl, label):

    try:
        
        reader = vtk.vtkNIFTIImageReader()
        reader.SetFileName(filename_nii)
        reader.Update()
        
        surf = vtk.vtkDiscreteMarchingCubes()
        surf.SetInputConnection(reader.GetOutputPort())
        surf.SetValue(0, label)
        surf.Update()
        
        smoother= vtk.vtkWindowedSincPolyDataFilter()
        if vtk.VTK_MAJOR_VERSION <= 5:
            smoother.SetInput(surf.GetOutput())
        else:
            smoother.SetInputConnection(surf.GetOutputPort())
        smoother.SetNumberOfIterations(30)
        smoother.NonManifoldSmoothingOn()
        smoother.NormalizeCoordinatesOn()
        smoother.GenerateErrorScalarsOn()
        smoother.Update()
         
        writer = vtk.vtkSTLWriter()
        writer.SetInputConnection(smoother.GetOutputPort())
        writer.SetFileTypeToASCII()
        writer.SetFileName(filename_stl)
        writer.Write()
        
    except:
        
        pass


if __name__ == "__main__":

    def working(X):

        # Task 2 : DBSCAN Segmentation (C++ implementation)
        cluster_label = str(X)[:2]

        os.mkdir('Clusters_' + str(cluster_label))

        print("\nFor this DBSCAN segmentation, epsilon value is 25, and minpoints value is 40. For eps and minpts, edit main.cpp if needed. For size thresholds, edit this script.")

        a = nib.load(str(X))
        A = np.array(a.dataobj)

        valid_coord = []

        for x in range(0,A.shape[0]-1):
            for y in range(0,A.shape[1]-1):
                for z in range(0,A.shape[2]-1):
                    if (A[x][y][z] == 1):
                        valid_coord += [[x,y,z]]

        np.savetxt("valid_coord.csv", valid_coord, delimiter=",")

        with open('valid_coord.csv', newline='') as csvfile:
            data = np.array(list(csv.reader(csvfile)))
            data_float = data.astype(float)

        add = np.array([[data_float.shape[0], ' ', ' ']])

        DBSCAN_prep = np.concatenate((add, data_float))

        np.savetxt("valid_coord.dat", DBSCAN_prep, fmt='%s', delimiter=',')

        os.remove("valid_coord.csv")

        df = pd.read_csv("valid_coord.dat", sep=";", header=0)

        df.rename(columns = lambda x: re.sub('\D','',x),inplace=True)

        df.to_csv("valid_coord_DBSCANprep.dat", sep = ' ', index = False)

        os.remove("valid_coord.dat")

        os.system("g++ main.cpp dbscan.cpp -o DBSCANcsv")

        os.system("./DBSCANcsv")

        os.remove("valid_coord_DBSCANprep.dat")

        # Cluster organization (cluster_label, Ashape, and valid_coord given from input)

        DBSCANraw = np.genfromtxt('DBSCAN_raw.csv')

        rownum = int(DBSCANraw.shape[0]/4 - 1)

        cluster_max = 0

        for i in range(rownum + 1):
            if DBSCANraw[4*i + 3] >= cluster_max:
                cluster_max = int(DBSCANraw[4*i + 3])
            
        cluster_lists = [[] for i in range(cluster_max + 1)]

        for i in range(rownum + 1):
            if DBSCANraw[4*i + 3] >= 1:
                cluster_lists[int(DBSCANraw[4*i + 3])].append([valid_coord[i]])
            
        for r in range(1,cluster_max + 1):
            cluster_indi = np.array(cluster_lists[r])
            cluster_coord = np.zeros((A.shape[0], A.shape[1], A.shape[2]))
            for s in range(len(cluster_indi)):
                cluster_coord[cluster_indi[s][0][0],cluster_indi[s][0][1],cluster_indi[s][0][2]] = 1
            if len(cluster_indi) >= 8000:
                cluster_nib = nib.Nifti1Image(cluster_coord, affine=np.eye(4))
                nib.save(cluster_nib, "DBSCAN-cluster" + str(r) + ".nii.gz")
                
                filename_nii = "DBSCAN-cluster" + str(r) + ".nii.gz"
                filename_stl = filename_nii[:-7] + '.stl'
                label = 1
                
                nii_2_mesh(filename_nii, filename_stl, label)
                    
                shutil.move("DBSCAN-cluster" + str(r) + ".nii.gz", 'Clusters_' + str(cluster_label))
                shutil.move("DBSCAN-cluster" + str(r) + ".stl", 'Clusters_' + str(cluster_label))

        os.remove("DBSCAN_raw.csv")
        shutil.move(str(X),'Clusters_' + str(cluster_label))

        os.system("zip -r /content/Clusters_" + str(cluster_label) + ".zip /content/Clusters_" + str(cluster_label))
        files.download("/content/Clusters_" + str(cluster_label) + ".zip")  

#working('05_T1w_post_SCALP_normalized_filt_nonMNI_difference_post=0.nii.gz')
#working('07_T1w_post_SCALP_normalized_filtered_nonMNI_difference_post=0.nii.gz')
working('10_T1w_post_SCALP_normalized_filtered_nonMNI_difference_post=0.nii.gz')
#working('13_T1w_post_SCALP_normalized_filtered_nonMNI_difference_post=0.nii.gz')
#working('16_T1w_post_SCALP_normalized_filtered_nonMNI_difference_post=0.nii.gz')
#working('20_T1w_post_SCALP_normalized_filtered_nonMNI_difference_post=0.nii.gz')
#working('25_T1w_post_SCALP_normalized_filtered_nonMNI_difference_post=0.nii.gz')
#working('26_T1w_post_SCALP_normalized_filtered_nonMNI_difference_post=0.nii.gz')
#working('28_T1w_post_SCALP_normalized_filtered_nonMNI_difference_post=0.nii.gz')
#working('28_T1w_post_SCALP_normalized_filtered_MNI_difference_post=0.nii.gz')




For this DBSCAN segmentation, epsilon value is 25, and minpoints value is 40. For eps and minpts, edit main.cpp if needed. For size thresholds, edit this script.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import files  

cluster_label = '28'

os.system("zip -r /content/Clusters_" + str(cluster_label) + ".zip /content/Clusters_" + str(cluster_label))
files.download("/content/Clusters_" + str(cluster_label) + ".zip")